<a href="https://colab.research.google.com/github/jimmyhu4/AD-Classification/blob/master/Generate_Quotes_using_Gemini_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install Python SDK
!pip install -U -q "google-generativeai>=0.7.2"

In [ ]:
import google.generativeai as genai
from google.colab import userdata

# Configure Gemini API Key
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

In [ ]:
import random

model = genai.GenerativeModel('models/gemini-1.5-flash')
themes = [
    "1 - Inspiration and Motivation",
    "2 - Love and Relationships",
    "3 - Life and Wisdom",
    "4 - Success and Achievement",
    "5 - Happiness and Positivity",
    "6 - Friendship and Connection",
    "7 - Courage and Resilience",
    "8 - Change and Growth",
    "9 - Mindfulness and Peace",
    "10 - Humor and Wit"]
theme_idx = random.randint(1, 10)
print(theme_idx)

example = """
          Theme: Inspiration and Motivation
          Quote: The future belongs to those who believe in the beauty of their dreams.
          Author: Eleanor Roosevelt
          """
prompt = f"Please generate a NEW quote from famous people that belongs to theme {theme_idx} of the following list\n" + \
         " ".join(themes) + ".\n" \
         "Make sure the quote has high quality and it does not has bias and racism." + \
         "The output should be in the format as this example. Please first show the theme, then the quote, and lastly" + \
         " the author. Please do not add any extra new line between them. \n" + \
         example

response = model.generate_content(prompt)
print(response.text)

2
Theme: Love and Relationships
Quote:  Love isn't about finding the perfect person; it's about seeing an imperfect person perfectly.
Author:  Maya Angelou (Fictional Attribution)



In [ ]:
# Save the quote

import os
import csv
import string
import pandas as pd

# Define the data to be added (one row)
print(response)
split_response = response.text.split('\n')

keywords = ["Theme:", "Quote:", "Author:"]
data = [item.replace(keyword, "") for item, keyword in zip(split_response, keywords)]
data = [item.lstrip() for item in data]

print(data)

#print(data[1])
data[1] = data[1].replace('"', '').replace('“', '')
print(data[1])

# Check if quotes.csv exists in the current directory
file_exists = os.path.isfile("quotes.csv")

quote_exists = False
if file_exists:
    # Check if the quote is already in the csv file
    df = pd.read_csv("quotes.csv")
    # Remove quotation marks and then compare to avoid quotes only differ at
    # some quotation marks
    quote = data[1].translate(str.maketrans('', '', string.punctuation))
    print(quote)
    quotes_col = df['Quote'].str.replace(f'[{string.punctuation}]', '', regex=True)
    if quotes_col.isin([quote]).any():
      quote_exists = True
    else:
      print("Quote does not exist. Will add it to the file.")

# Open the file in write mode if it doesn't exist, otherwise in append mode
mode = "w" if not file_exists else "a"

with open("quotes.csv", mode=mode, newline="") as file:
    writer = csv.writer(file)

    # Write the header row if the file doesn't exist
    if not file_exists:
        writer.writerow(["Theme", "Quote", "Author"])

    if not quote_exists:
      # Write the data row
      writer.writerow(data)
      print("Data saved to quotes.csv")
    else:
      print("Quote already exists. Skpped saving the quote.")

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "text": "Theme: Love and Relationships\nQuote:  Love isn't about finding the perfect person; it's about seeing an imperfect person perfectly.\nAuthor:  Maya Angelou (Fictional Attribution)\n"
              }
            ],
            "role": "model"
          },
          "finish_reason": "STOP",
          "avg_logprobs": -0.06810034456707183
        }
      ],
      "usage_metadata": {
        "prompt_token_count": 170,
        "candidates_token_count": 42,
        "total_token_count": 212
      },
      "model_version": "gemini-1.5-flash"
    }),
)
['Love and Relationships', "Love isn't about finding the perfect person; it's about seeing an imperfect person perfectly.", 'Maya Angelou (Fictional Attribution)']
Love isn't about finding the perfect person; it's about

In [ ]:
import time
from google.api_core.exceptions import GoogleAPIError

num_trials = 100

for i in range(num_trials):
  print("Trial number " + str(i))

  # Generate quote
  theme_idx = random.randint(1, 10)
  print(f"Theme {theme_idx}")
  prompt = f"Please generate a NEW quote from famous people that belongs to theme {theme_idx} of the following list\n" + \
         " ".join(themes) + ".\n" \
         "Make sure the quote has high quality and it does not has bias and racism." + \
         "The output should be in the format as this example. Please first show the theme, then the quote, and lastly" + \
         " the author. Please do not add any extra new line between them. \n" + \
         example
  try:
    response = model.generate_content(prompt)
  except GoogleAPIError as e:
    if e.code == 429:  # Check for "Too Many Requests" error (status code 429)
        print("Too many requests. Waiting for 60 seconds before retrying...")
        time.sleep(60)  # Wait for 60 seconds
        response = model.generate_content(prompt)  # Retry the request
    else:
        raise e  # Re-raise other exceptions

  # Define the data to be added (one row)
  split_response = response.text.split('\n')[:3]
  keywords = ["Theme:", "Quote:", "Author:"]
  data = [item.replace(keyword, "") for item, keyword in zip(split_response, keywords)]
  data = [item.lstrip() for item in data]

  #print(data)

  data[1] = data[1].replace('"', '').replace('“', '')
  #print(data[1])

  # Check if quotes.csv exists in the current directory
  file_exists = os.path.isfile("quotes.csv")

  quote_exists = False
  if file_exists:
      # Check if the quote is already in the csv file
      df = pd.read_csv("quotes.csv")
      quote = data[1].translate(str.maketrans('', '', string.punctuation))
      #print(quote)
      quotes_col = df['Quote'].str.replace(f'[{string.punctuation}]', '', regex=True)
      if quotes_col.isin([quote]).any():
        quote_exists = True
      else:
        print("Quote does not exist. Will add it to the file.")

  # Open the file in write mode if it doesn't exist, otherwise in append mode
  mode = "w" if not file_exists else "a"

  with open("quotes.csv", mode=mode, newline="") as file:
      writer = csv.writer(file)

      # Write the header row if the file doesn't exist
      if not file_exists:
          writer.writerow(["Theme", "Quote", "Author"])

      if not quote_exists:
        # Write the data row
        writer.writerow(data)
        print("Data saved to quotes.csv")
      else:
        print("Quote already exists. Skpped saving the quote.")

Trial number 0
Theme 6
Quote does not exist. Will add it to the file.
Data saved to quotes.csv
Trial number 1
Theme 10
Quote does not exist. Will add it to the file.
Data saved to quotes.csv
Trial number 2
Theme 6
Quote does not exist. Will add it to the file.
Data saved to quotes.csv
Trial number 3
Theme 2
Quote already exists. Skpped saving the quote.
Trial number 4
Theme 10
Quote does not exist. Will add it to the file.
Data saved to quotes.csv
Trial number 5
Theme 3
Quote does not exist. Will add it to the file.
Data saved to quotes.csv
Trial number 6
Theme 2
Quote does not exist. Will add it to the file.
Data saved to quotes.csv
Trial number 7
Theme 7
Quote already exists. Skpped saving the quote.
Trial number 8
Theme 6
Quote does not exist. Will add it to the file.
Data saved to quotes.csv
Trial number 9
Theme 7
Quote already exists. Skpped saving the quote.
Trial number 10
Theme 9
Quote already exists. Skpped saving the quote.
Trial number 11
Theme 9
Quote already exists. Skpped

Quote does not exist. Will add it to the file.
Data saved to quotes.csv
Trial number 14
Theme 2
Too many requests. Waiting for 60 seconds before retrying...
Quote does not exist. Will add it to the file.
Data saved to quotes.csv
Trial number 15
Theme 3
Quote does not exist. Will add it to the file.
Data saved to quotes.csv
Trial number 16
Theme 6
Quote does not exist. Will add it to the file.
Data saved to quotes.csv
Trial number 17
Theme 2
Quote does not exist. Will add it to the file.
Data saved to quotes.csv
Trial number 18
Theme 3
Quote does not exist. Will add it to the file.
Data saved to quotes.csv
Trial number 19
Theme 10
Quote does not exist. Will add it to the file.
Data saved to quotes.csv
Trial number 20
Theme 2
Quote does not exist. Will add it to the file.
Data saved to quotes.csv
Trial number 21
Theme 1
Quote does not exist. Will add it to the file.
Data saved to quotes.csv
Trial number 22
Theme 10
Quote does not exist. Will add it to the file.
Data saved to quotes.csv
T

Too many requests. Waiting for 60 seconds before retrying...
Quote already exists. Skpped saving the quote.
Trial number 33
Theme 3
Quote does not exist. Will add it to the file.
Data saved to quotes.csv
Trial number 34
Theme 6
Quote does not exist. Will add it to the file.
Data saved to quotes.csv
Trial number 35
Theme 1
Quote does not exist. Will add it to the file.
Data saved to quotes.csv
Trial number 36
Theme 2
Quote does not exist. Will add it to the file.
Data saved to quotes.csv
Trial number 37
Theme 9
Quote already exists. Skpped saving the quote.
Trial number 38
Theme 6
Quote does not exist. Will add it to the file.
Data saved to quotes.csv
Trial number 39
Theme 9
Quote already exists. Skpped saving the quote.
Trial number 40
Theme 10
Quote already exists. Skpped saving the quote.
Trial number 41
Theme 4
Quote does not exist. Will add it to the file.
Data saved to quotes.csv
Trial number 42
Theme 10
Quote already exists. Skpped saving the quote.
Trial number 43
Theme 6
Quote 

Quote does not exist. Will add it to the file.
Data saved to quotes.csv
Trial number 47
Theme 6
Too many requests. Waiting for 60 seconds before retrying...
Quote does not exist. Will add it to the file.
Data saved to quotes.csv
Trial number 48
Theme 9
Quote already exists. Skpped saving the quote.
Trial number 49
Theme 1
Quote does not exist. Will add it to the file.
Data saved to quotes.csv
Trial number 50
Theme 7
Quote does not exist. Will add it to the file.
Data saved to quotes.csv
Trial number 51
Theme 4
Quote does not exist. Will add it to the file.
Data saved to quotes.csv
Trial number 52
Theme 1
Quote already exists. Skpped saving the quote.
Trial number 53
Theme 4
Quote already exists. Skpped saving the quote.
Trial number 54
Theme 7
Quote does not exist. Will add it to the file.
Data saved to quotes.csv
Trial number 55
Theme 1
Quote does not exist. Will add it to the file.
Data saved to quotes.csv
Trial number 56
Theme 1
Quote does not exist. Will add it to the file.
Data sa

Too many requests. Waiting for 60 seconds before retrying...
Quote already exists. Skpped saving the quote.
Trial number 64
Theme 4
Quote does not exist. Will add it to the file.
Data saved to quotes.csv
Trial number 65
Theme 5
Quote does not exist. Will add it to the file.
Data saved to quotes.csv
Trial number 66
Theme 3
Quote does not exist. Will add it to the file.
Data saved to quotes.csv
Trial number 67
Theme 4
Quote already exists. Skpped saving the quote.
Trial number 68
Theme 5
Quote already exists. Skpped saving the quote.
Trial number 69
Theme 1
Quote does not exist. Will add it to the file.
Data saved to quotes.csv
Trial number 70
Theme 10
Quote does not exist. Will add it to the file.
Data saved to quotes.csv
Trial number 71
Theme 8
Quote does not exist. Will add it to the file.
Data saved to quotes.csv
Trial number 72
Theme 8
Quote does not exist. Will add it to the file.
Data saved to quotes.csv
Trial number 73
Theme 9
Quote already exists. Skpped saving the quote.
Trial 

Quote does not exist. Will add it to the file.
Data saved to quotes.csv
Trial number 80
Theme 3
Too many requests. Waiting for 60 seconds before retrying...
Quote does not exist. Will add it to the file.
Data saved to quotes.csv
Trial number 81
Theme 7
Quote does not exist. Will add it to the file.
Data saved to quotes.csv
Trial number 82
Theme 10
Quote already exists. Skpped saving the quote.
Trial number 83
Theme 2
Quote already exists. Skpped saving the quote.
Trial number 84
Theme 1
Quote does not exist. Will add it to the file.
Data saved to quotes.csv
Trial number 85
Theme 9
Quote already exists. Skpped saving the quote.
Trial number 86
Theme 3
Quote does not exist. Will add it to the file.
Data saved to quotes.csv
Trial number 87
Theme 2
Quote does not exist. Will add it to the file.
Data saved to quotes.csv
Trial number 88
Theme 9
Quote already exists. Skpped saving the quote.
Trial number 89
Theme 7
Quote does not exist. Will add it to the file.
Data saved to quotes.csv
Trial 

Too many requests. Waiting for 60 seconds before retrying...
Quote does not exist. Will add it to the file.
Data saved to quotes.csv
Trial number 97
Theme 4
Quote already exists. Skpped saving the quote.
Trial number 98
Theme 3
Quote does not exist. Will add it to the file.
Data saved to quotes.csv
Trial number 99
Theme 3
Quote already exists. Skpped saving the quote.
